In [2]:
import numpy as np
import time, torch
import torch.nn.functional as F
from collections import OrderedDict
from torch import nn, optim
from torchvision import datasets, transforms
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])
# Download and load the training data
trainset = datasets.MNIST('/home/yungshun/workspace/py3/pytorch-mnist-dnn-classifier/datasets/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Processing...
Done!


In [4]:
# Hyperparameters for our network
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Build a feed-forward network
model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(input_size, hidden_sizes[0])),
    ('relu1', nn.ReLU()),
    ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
    ('relu2', nn.ReLU()),
    ('logits', nn.Linear(hidden_sizes[1], output_size))
]))